<a href="https://colab.research.google.com/github/IsaacSGray/L02twopOpt/blob/main/Copy_of_L01_Greedy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [L01: Implementing Greedy TSP](https://docs.google.com/document/d/1JA_CWBXPN6whvJAemnJLvtYJ3Pbk84dTUolWO_QqosA/edit?usp=sharing)
Written by Dr. Jan Pearce, Berea College

Complete the code by meeting all linked requirements and fixing all FIXMEs

## Your Name: Isaac Gray

**SUMMARY**: A brief summary description of the design and implementation, including how much your initial design plan evolved, the final result you achieved and the amount of time you spent as a programmer or programmers in accomplishing these results. This should be no more than two paragraphs.

I started with the extraction method, then a list of places method. Both were important to my design process, as they give me my data to workwith. My design with the extraction of the data I devised after some lab hours with nakazawa and research with splicing of list in python, and some stack overflow on maps.

After that I started with setting up my coordinate data in self.coords. I loop over self.nodedata which contains everything from its ID and all other important data. With some research into accessing specific elements to refresh myself on my python, I get it working. Accessing and appending each nods x and y coordinate. Useful for calculating distance.

Which is what I did next. To calculate the distance of the route was quite useful as it lets us later implement a method where we need to find the nearest city. I did some stackoverflow search on calculating distance with networkx graphs, plus some gernative AI make this formula I used easier for me to understand move on my development.

Geroute was the hardest part of development given with how many variables and just keeping track of when and where we are in the route. I started by checking if there are places to visit, then while there are places to check, clean up the list of places visited, and check if its not in visited. I used a fancy lil trick gathered from stack overflow and also generativeAI on using the min function with a lamda expression. It was confusing and took some debugging with syntax.

I designed getGreedy last by simply calling the route function and appending the starting city to our route.

**PERSONNEL**: Met with nakazawa at the start, consulted generative AI, stack overflow, and W3schools on anything else that didn't make sense to me. Some questions I had were also answered by peers in the course (those who had more recent python experience than I)

**CHALLENGES**: What was the most challenging was the datastructures, some nuance python functions, and just *how* to imagine the graph. A huge issue I have is needing to conceptualize, visualize, and solve the problem conceptually before I can do it. I spent a lot of time on the drawing board before I put a single character in a line of code. Another issue was the link not working. It was sort of a mega headache. After I got past that at that it was a bit smooth other than trying to read the list- which was an issue since it was spaced out, unformatted.

Another challenge was how to go about calculating distance, plus how to find the nearest city- that was a massive freaking headache for me as I didnt quite have my route yet, so calculation from current node was hard for me to conceptualize- so it held me back for a while.

**INNOVATIONS**: I did not make any fancy or new innovations that were not asked of me. I did was necessary to complete the document. The most 'innovative' was fixing the URL document, and the calculate distance plus min methods used.

**TESTING**: I tested things block by block, solving the smaller problems at first and testing them- and the bigger problems that had bigger collateral to debugging and testing I tested last as they often relied on directly or indirectly the smaller methods

**ERRORS**: URL not working properly, that was frustrating as heck.

**COMMENTS**: This was a pain in my butt- but that was due to me being crazy sick at the time of development and me solving the problem. I took an incremental approach to design and development and that really helped me. A huge comment I'd have to make is just how much help nakazawa was at the start with resolving thie mystical link not working properly. Another great help was generative AI which helped me with some niche python functions and methods that I was unfamiliar with.

I absolutely abhor networkx due to its documentation being a bit..cluttered and confusing- and with research- also outdated.



## Import Libraries

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import math
from urllib.request import urlopen

### Read data

In [ ]:
tspdata = urlopen("http://cs.berea.edu/courses/csc445/data/tsp-small.txt").read().decode('ASCII')
print(tspdata)

## TSP Class

In [ ]:
"""

Given certain input, it uses the greedy algorithm to plot a graph and output a file of the solution path
"""

class TSP:

    def __init__(self):
        """ TSP class containing data needed for networkx graph creation, file reading and file writing.
            Creates a graph for the traveling salesperon problem using the greedy algorithm
        """

        self.dimension = None
        self.nodeData = []
        self.graph = nx.DiGraph()
        self.coords = {}
        self.places = []
        self.route = []

    def extractData(self, data):
        """Extracts the number of vertices and their coordinates from the input data.
        """
        lines = data.strip().split('\n')
        self.dimension = int(lines[0].strip())  # First line: number of nodes (vertices)
        for line in lines[1:]:
            x, y = map(int, line.strip().split())  # Remaining lines: x and y coordinates
            self.nodeData.append((len(self.nodeData) + 1, x, y))  # store the node and where it is on the graph


  def setUpCoords(self):
        """Sets up the coordinates for each node in a dictionary as tuples.
        """
       # goes over each node in the list `self.nodeData`
        for node in self.nodeData:
            node_id = node[0] #first element is nodes int ID
            xcord = node[1] #second element is the x cord of the node
            ycord = node[2] #third element is the y cord of the node
            self.coords[node_id] = (xcord, ycord) # Store the coordinates  as a tuple in the dictionary `self.coords`with the node ID as the key

    def calculateDistance(self, a, b):
        """Calculates the distance between points a and then points b.
         Arguments:
        a (tuple): The coordinates of the first point as (x, y).
        b (tuple): The coordinates of the second point as (x, y).

    Returns:
        float: The distance between the two points using the distance formula
        """
        return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2) # first,calculates the squared the difference between the x cords,
       # then, it calculates the squared the difference between the y cords, and then finally
       # square root the sum of the differences squared to get final distance

    def getRoute(self, current, visits=[], nVisits=[], needList=True):
        """Computes the route using the greedy algorithm by making the list of nodes to be visited.
        Arguments:
        current (int): the id of the current node (starting point).
        visits (list) list of nodes that have been visited.
        nVisits (list): list of nodes that have not been visited. .
        needList (bool): flag indicating whether to initialize the list of nodes to visit.

    """
        if needList:
            nVisits = self.listPlaces()# starts the list of nodes that havent been visited
        while nVisits:
            visits.append(current) # adds the current node to the list of visited nodes
            nVisits = self.cleanUpList(visits, nVisits)# updates the list of unvisited nodes by removing the nodes that have been visited
            if not nVisits:# No more nodes to visit
                break #break if there are no more visits to make
            # finds the closest city
            nearest_city = min(nVisits, key=lambda x: self.calculateDistance(self.coords[current], self.coords[x]))
            #search nvisits, our iterable,our expression x is calculatingDistance from self.coords on the current node, this will be the
            #node that is closest to us from the current node.
            self.route.append(nearest_city)
            current = nearest_city          # updates the current node to be the nearest node
        # Ensure that the route is completed fully
        if visits and visits[0] != current: #if our visits list, and last element in visits is not our current node, then append visits[0] to route.
            self.route.append(visits[0])

    def beGreedy(self, start):
        """Executes the greedy algorithm to find the route.
    Arguments:
        start (int): The ID of the starting node for the route.
        Pair prog: Clau was the Driver and Sara was the navigator
    """
        self.route = [start] # we append our start node to the route
        self.getRoute(start) # call ger route from the start function
        self.route.append(start) #we make sure our last city in the list is our starting city

    def listPlaces(self):
        """Creates a list of all node ids from nodeData.
        """
        return [node[0] for node in self.nodeData] #grabs the integer ID of each node in a list.

    def cleanUpList(self, visited, notVisited):
        """removes items in visited from notVisited
        Arguments:
            visited {list}
            notVisited {list}
        Returns:
            [list] -- a list of notVisited - visited
        """

        for item in visited:
            if item in notVisited:
                notVisited.remove(item)
        return notVisited

    def addNodes(self):
        """ adds nodes to the networkx graph
        """

        for everything in self.nodeData:
            node = everything[0]
            self.graph.add_node(node, pos=self.coords[node])

    def addEdges(self):
        """Adds graphs to the networkx graph
        """

        x = 0
        while x < len(self.route)-1:
            self.graph.add_edge(self.route[x], self.route[x+1],
                                weight=self.calculateDistance(self.coords[self.route[x]], self.coords[self.route[x+1]]))
            x += 1

    def showGraph(self):
        """Uses the imported libraries to display the graph
        """
        # get labels and coordinates for the graph
        coords = nx.get_node_attributes(self.graph, 'pos')
        labels = nx.get_edge_attributes(self.graph, 'weight')
        # draw nodes in their coordinates
        nx.draw(self.graph, coords)
        nx.draw_networkx_labels(self.graph, coords)
        # draw edges with their weights
        nx.draw_networkx_edge_labels(self.graph, coords, edge_labels=labels)
        plt.show()




def main():
    tsp = TSP()

    tsp.extractData(tspdata)
    tsp.setUpCoords()
    tsp.addNodes()
    tsp.beGreedy(1)
    tsp.addEdges()
    tsp.showGraph()


main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 40)